In [1]:
import matplotlib
matplotlib.use("Agg")
# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
import os
try:
    os.mkdir('/tmp/cell-images1')
    os.mkdir('/tmp/cell-images1/training')
    os.mkdir('/tmp/cell-images1/training/parasitized')
    os.mkdir('/tmp/cell-images1/training/uninfected')
    os.mkdir('/tmp/cell-images1/training-T')
    os.mkdir('/tmp/cell-images1/training-T/parasitized')
    os.mkdir('/tmp/cell-images1/training-T/uninfected')
    os.mkdir('/tmp/cell-images1/validation')
    os.mkdir('/tmp/cell-images1/validation/parasitized')
    os.mkdir('/tmp/cell-images1/validation/uninfected')
    os.mkdir('/tmp/cell-images1/testing')
    os.mkdir('/tmp/cell-images1/testing/parasitized')
    os.mkdir('/tmp/cell-images1/testing/uninfected')
    #YOUR CODE GOES HERE
except OSError:
    print("Error")
    pass



In [24]:
import random
from shutil import copyfile
def split_data(SOURCE, TRAINING, TESTING,SPLIT_SIZE):
    x=0
    y=os.listdir(SOURCE)[0:10000]
    random.sample(y,len(y))
    for i in y:
      final=os.path.join(SOURCE,i)
      if x<(SPLIT_SIZE)*len(y):
        copy=os.path.join(TRAINING,i)
        if os.path.getsize(final)==0:
          print(str(i)+" is zero length, so ignoring")
        else:
          copyfile(final,copy)
      else:
        copy=os.path.join(TESTING,i)
        if os.path.getsize(final)==0:
          print(str(i)+" is zero length, so ignoring")
        else:
          copyfile(final,copy)
      x=x+1

P_SOURCE_DIR = "/content/drive/MyDrive/cell_images/Parasitized"
TRAINING_P_DIR = '/tmp/cell-images1/training/parasitized'
TRAINING_PS_DIR = '/tmp/cell-images1/training-T/parasitized'
TESTING_P_DIR = '/tmp/cell-images1/testing/parasitized'
VALIDATION_P_DIR = '/tmp/cell-images1/validation/parasitized'
U_SOURCE_DIR = "/content/drive/MyDrive/cell_images/Uninfected"
TRAINING_US_DIR = '/tmp/cell-images1/training-T/uninfected'
TRAINING_U_DIR = '/tmp/cell-images1/training/uninfected'
TESTING_U_DIR = '/tmp/cell-images1/testing/uninfected'
VALIDATION_U_DIR= '/tmp/cell-images1/validation/uninfected'


split_size = .8
split_data(P_SOURCE_DIR, TRAINING_PS_DIR, TESTING_P_DIR,split_size)
split_data(U_SOURCE_DIR, TRAINING_US_DIR, TESTING_U_DIR,split_size)
split_data(TRAINING_PS_DIR, TRAINING_P_DIR, VALIDATION_P_DIR,split_size)
split_data(TRAINING_US_DIR, TRAINING_U_DIR, VALIDATION_U_DIR,split_size)



In [25]:
NUM_EPOCHS = 50
INIT_LR = 1e-1
BS = 32
def poly_decay(epoch):
	maxEpochs = NUM_EPOCHS
	baseLR = INIT_LR
	power = 1.0
	alpha = baseLR * (1 - (epoch / float(maxEpochs))) ** power
	return alpha

In [22]:
!rm -rf /tmp/cell-images1

In [26]:
trainAug = ImageDataGenerator(
	rescale=1 / 255.0,
	rotation_range=20,
	zoom_range=0.05,
	width_shift_range=0.05,
	height_shift_range=0.05,
	shear_range=0.05,
	horizontal_flip=True,
	fill_mode="nearest")


valAug = ImageDataGenerator(rescale=1 / 255.0)

In [27]:
trainGen = trainAug.flow_from_directory(
	"/tmp/cell-images1/training",
	class_mode="categorical",
	target_size=(64, 64),
	color_mode="rgb",
	shuffle=True,
	batch_size=BS)
# initialize the validation generator
valGen = valAug.flow_from_directory(
	"/tmp/cell-images1/validation",
	class_mode="categorical",
	target_size=(64, 64),
	color_mode="rgb",
	shuffle=False,
	batch_size=BS)
# initialize the testing generator
testGen = valAug.flow_from_directory(
	"/tmp/cell-images1/testing",
	class_mode="categorical",
	target_size=(64, 64),
	color_mode="rgb",
	shuffle=False,
	batch_size=BS)

Found 12798 images belonging to 2 classes.
Found 3200 images belonging to 2 classes.
Found 4000 images belonging to 2 classes.


In [28]:
model = ResNet50(weights=None,classes=2)
opt = SGD(lr=INIT_LR, momentum=0.9)
model.compile(loss="categorical_crossentropy", optimizer=opt,	metrics=["accuracy"])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [30]:
callbacks = [LearningRateScheduler(poly_decay)]
totalTrain=len(os.listdir("/tmp/cell-images1/training/parasitized")) + len(os.listdir("/tmp/cell-images1/training/uninfected"))
totalVal=len(os.listdir("/tmp/cell-images1/validation/parasitized")) + len(os.listdir("/tmp/cell-images1/validation/uninfected"))
print(totalTrain)
H = model.fit(
          	x=trainGen,
            steps_per_epoch=totalTrain // BS,
	          validation_data=valGen,
	          validation_steps=totalVal // BS,
	          epochs=50,
            callbacks=callbacks)

12800
Epoch 1/50
400/400 [==============================] - 43s 107ms/step - loss: 0.3677 - accuracy: 0.8534 - val_loss: 0.3655 - val_accuracy: 0.8750
Epoch 2/50
400/400 [==============================] - 43s 108ms/step - loss: 0.3770 - accuracy: 0.8544 - val_loss: 0.4467 - val_accuracy: 0.9159
Epoch 3/50
400/400 [==============================] - 44s 111ms/step - loss: 0.2569 - accuracy: 0.9012 - val_loss: 0.2641 - val_accuracy: 0.9319
Epoch 4/50
400/400 [==============================] - 43s 107ms/step - loss: 0.2320 - accuracy: 0.9146 - val_loss: 0.2665 - val_accuracy: 0.9369
Epoch 5/50
400/400 [==============================] - 45s 112ms/step - loss: 0.2128 - accuracy: 0.9230 - val_loss: 0.2306 - val_accuracy: 0.9441
Epoch 6/50
400/400 [==============================] - 42s 106ms/step - loss: 0.1970 - accuracy: 0.9317 - val_loss: 0.2534 - val_accuracy: 0.9463
Epoch 7/50
400/400 [==============================] - 44s 110ms/step - loss: 0.1932 - accuracy: 0.9315 - val_loss: 0.4403 - 

In [31]:
totalTest=len(os.listdir("/tmp/cell-images/testing"))
testGen.reset()
predIdxs = model.predict(x=testGen)
predIdxs = np.argmax(predIdxs, axis=1)
print(classification_report(testGen.classes, predIdxs,	target_names=testGen.class_indices.keys()))


              precision    recall  f1-score   support

 parasitized       0.94      0.90      0.92      2000
  uninfected       0.90      0.94      0.92      2000

    accuracy                           0.92      4000
   macro avg       0.92      0.92      0.92      4000
weighted avg       0.92      0.92      0.92      4000



In [32]:
model.save("malaria-detection-model.h5")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


/content
